In [58]:
import time

import torch
from torch.utils.data import DataLoader

from torchtext import datasets
from torchtext.data import Field
from torchtext.data.utils import get_tokenizer

from transformers import AutoTokenizer

from utils import extract_config
from constants import *

import lineflow.datasets as lfds
from transformer_pl import *
from data_pl import *

# config
config = {
    "embedding_dimension": 200,
    "ff_dimension": 200,
    "n_attention_heads": 2,
    "n_encoder_layers": 0,
    "n_decoder_layers": 2,
    "dataset": Dataset.WikiText103.name,
    "segmentation": Segmentation.Word.name,
    "max_seq_len": 35,
    "batch_size": 20,
    "eval_batch_size": 10,
    "dropout": 0.2,
    "n_epochs": 3,
    "learning_rate": 0.0001,
    "adam_b1": 0.9,
    "adam_b2": 0.999,
    "adam_l2_weightdecay": 0.01,
    "loss_criterion": "CrossEntropyLoss"
}

# setup
train_dataloader, val_dataloader, test_dataloader, vocab = load_data_word(config)
ntokens = len(vocab.stoi)

# init model
model = DecoderOnlyTransformer(config, ntokens)
trainer = pl.Trainer(gpus=2)
trainer.fit(model, train_dataloader, val_dataloader)

[Start Load Data]
Fetched Data (0.000009s)
downloading wikitext-103-v1.zip
wikitext-103-v1.zip: 100%|██████████| 190M/190M [00:04<00:00, 38.6MB/s]
extracting
Tokenized and Split Data (67.779651s)
Built Vocab (87.863547s)
torch.Size([5167283, 20])
torch.Size([10916, 20])
torch.Size([12320, 20])
[End Load Data] (1488.603157s)


wandb: wandb version 0.10.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


MisconfigurationException: 
                You requested GPUs: [0, 1]
                But your machine only has: []
            

In [1]:
from data_pl import *
from utils import *
config = {
    "embedding_dimension": 200,
    "ff_dimension": 200,
    "n_attention_heads": 2,
    "n_encoder_layers": 0,
    "n_decoder_layers": 2,
    "dataset": Dataset.PennTreebank.name,
    "segmentation": Segmentation.Word.name,
    "max_seq_len": 35,
    "batch_size": 20,
    "eval_batch_size": 10,
    "dropout": 0.2,
    "n_epochs": 3,
    "learning_rate": 0.0001,
    "adam_b1": 0.9,
    "adam_b2": 0.999,
    "adam_l2_weightdecay": 0.01,
    "loss_criterion": "CrossEntropyLoss"
}

train_dataloader, val_dataloader, test_dataloader, vocab = load_data(config)
# print(vocab.stoi)

In [9]:
for batch in train_dataloader:
    data, targets = batch
    print("data", data.shape)
    print("targets",targets.shape)
    break



print(emb_to_string(data[0], vocab))
print(emb_to_string(targets[0:20], vocab))

data torch.Size([35, 20])
targets torch.Size([700])
aer of <eos> n capital experience mr they soviet all the yeast incentives <unk> were because awarded <eos> hong were
banknote them <unk> billion cities\/abc made . say state the student <eos> and up a of to if kong going


In [8]:
targets[0:20]

tensor([6138,  132,    0,   53, 6532,  167,   10,  121,  158,    2, 2341,    3,
           9,   56,    7,    5,    6,   73,  793,  320])

In [57]:
from transformers import GPT2TokenizerFast



print("[Start Load Data]")
ts = time.time()

# get dataset
dataset, batch_size, max_seq_len = extract_config(
    config, "dataset", "batch_size", "max_seq_len")
dataset = getattr(datasets, dataset)
print(f"Fetched Data ({time.time() - ts:3f}s)")

# tokenize
pretrained_weights = 'gpt2'
tokenizer_en = GPT2TokenizerFast.from_pretrained(pretrained_weights)
tokenizer_en.pad_token = tokenizer_en.eos_token

# split dataset
train_dataset, val_dataset, test_dataset = dataset.splits(
    text_field=field_processor)
print(f"Tokenized and Split Data ({time.time() - ts:3f}s)")

# # get vocabulary
# vocab = tokenizer_en.get_vocab()

# # data prep
# def data_prep(tt_dataset_split):
#     raw_text_iter = tt_dataset_split[0].text
#     data = [tokenizer_en.encode(item) for item in raw_text_iter]
#     data = torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))
#     # Divide the dataset into bsz parts.
#     nbatch = data.size(0) // batch_size
#     # Trim off any extra elements that wouldn't cleanly fit (remainders).
#     data = data.narrow(0, 0, nbatch * batch_size)
#     # Evenly divide the data across the batch_size batches.
#     data = data.view(batch_size, -1).t().contiguous()
#     return data

# # # setup dataloaders
# train_dataloader = TextDataloader(data_prep(train_dataset), max_seq_len)
# # val_dataloader = TextDataloader(data_prep(val_dataset), max_seq_len)
# # test_dataloader = TextDataloader(data_prep(test_dataset), max_seq_len)

[Start Load Data]
Fetched Data (0.000145s)
Tokenized and Split Data (2.395858s)


AttributeError: 'list' object has no attribute 'numel'

In [55]:
train_dataloader

[318]

In [31]:
print('---------- vocab ----------')
print()

print('vocab_files_names:',tokenizer_en.vocab_files_names)
print()

for k,v in tokenizer_en.pretrained_vocab_files_map.items():
    print(k)
    for kk,vv in v.items():
        print('- ',kk,':',vv)
    print()
    
print('vocab_size:',tokenizer_en.vocab_size)
print()
print(tokenizer_en.get_vocab())

# num = 50
# print(f'First {num} items of the vocab: {dict(itertools.islice(tokenizer_en.get_vocab().items(), 20))}')

ĠSyracuse': 29141, 'emia': 22859, '319': 35175, 'Ġstripe': 39858, 'Ġculinary': 35956, 'Ġpayout': 40055, 'Ġchain': 6333, 'Ġappealed': 22280, 'Ġparser': 30751, 'Ġtwentieth': 29112, 'ĠConor': 41708, 'Ġtheorem': 44728, 'onom': 6326, 'ĠStudio': 11733, 'tx': 17602, 'Ġreef': 25088, 'ĠWaves': 42733, 'ok': 482, 'ĠLegislature': 20815, '752': 43665, 'ĠSTA': 47550, 'raf': 32188, '1920': 40454, 'Ġorbiting': 40462, 'ĠLimit': 27272, 'æĺ': 23626, 'knife': 48810, 'Phone': 6132, 'ĠMedia': 6343, 'ĠBh': 16581, 'Isn': 41451, 'Bee': 49512, 'ĠOpposition': 29560, 'ĠProcessor': 32893, 'ĠBere': 37951, 'ĠBounce': 41474, 'ĠYou': 921, 'Ġinstitutional': 15855, 'ĠRequest': 19390, 'ĠPig': 23097, 'Ġsqu': 2809, 'ifty': 24905, 'Ġeldest': 34331, 'ĠThieves': 40392, 'Detroit': 40404, 'Amazing': 42770, 'ISC': 37719, '276': 27988, 'Be': 3856, 'Ġpracticable': 44791, 'Ġammunition': 14271, 'Ġacronym': 38787, 'quartered': 42508, 'Ġnetworks': 7686, 'dit': 5266, 'otype': 8690, 'med': 1150, 'ĠChoose': 17489, 'Ġstakes': 21147, 'Pand

In [28]:
from transformers import GPT2TokenizerFast

pretrained_weights = 'gpt2'
tokenizer_en = GPT2TokenizerFast.from_pretrained(pretrained_weights)
tokenizer_en.pad_token = tokenizer_en.eos_token


In [48]:
print("[Start Load Data]")
    ts = time.time()

    # get dataset
    dataset, batch_size, max_seq_len = extract_config(config, "dataset", "batch_size", "max_seq_len")
    dataset = getattr(datasets, dataset)
    print(f"Fetched Data ({time.time() - ts:3f}s)")

    # # tokenize
    tokenizer = get_tokenizer('basic_english')
    field_processor = Field(tokenize=tokenizer)

    # split dataset
    train_dataset, val_dataset, test_dataset = dataset.splits(
        text_field=field_processor)
    print(f"Tokenized and Split Data ({time.time() - ts:3f}s)")

    # get vocabulary
    field_processor.build_vocab(
        train_dataset, val_dataset, test_dataset, min_freq=1)
    vocab = field_processor.vocab
    print(f"Built Vocab ({time.time() - ts:3f}s)")

    # data prep
    def data_prep(tt_dataset_split):
        raw_text_iter = tt_dataset_split[0].text
        data = [torch.tensor([vocab[token] for token in tokenizer(item)],
                                dtype=torch.long) for item in raw_text_iter]
        data = torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))
        # Divide the dataset into bsz parts.
        nbatch = data.size(0) // batch_size
        # Trim off any extra elements that wouldn't cleanly fit (remainders).
        data = data.narrow(0, 0, nbatch * batch_size)
        # Evenly divide the data across the batch_size batches.
        data = data.view(batch_size, -1).t().contiguous()
        return data

    # setup dataloaders
    train_dataloader = TextDataloader(data_prep(train_dataset), max_seq_len)
    val_dataloader = TextDataloader(data_prep(val_dataset), max_seq_len)
    test_dataloader = TextDataloader(data_prep(test_dataset), max_seq_len)


[Start Load Data]
Fetched Data (0.000140s)
Tokenized and Split Data (1.490730s)
Built Vocab (1.899769s)
torch.Size([104335, 20])
torch.Size([10908, 20])
torch.Size([12310, 20])


In [ ]:
tokenizer_en.get_vocab()